In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
from scipy.ndimage.measurements import label
import time
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro 1/'
dir3='/Volumes/My Passport/'

In [ ]:
#####WITHOUT SIZE CONDITION TO CHECK FOR ELIMINATION

#yrs=np.arange(1940,1941,1)
yrs=np.arange(2004,2020,1)
num_total=0
num_new=0
for i in range(len(yrs)):
    year=yrs[i]
    print(year) 
    #ds=xr.open_dataset(dir2+'era_5_ar_detection_polar_'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra_2_ar_detection_'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era_5_ar_detection_test_'+str(year)+'.nc')
    
    #ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    
    ds1=xr.open_dataset(dir_data+'jra55gw.ar.labels.model.'+str(year)+'.nc')
    
    
    for j in range(ds.time.size):
        ar=ds['ar'][j,:,:]
 
        lons_right=[x for x in ar.lon.values if x>=60.]
        lons_left=[x for x in ar.lon.values if x<-60.]
        ar_right=ar.sel(lon=lons_right)
        ar_left=ar.sel(lon=lons_left)
        ar_right['lon']=[x-360. for x in lons_right]
        ar_left['lon']=[x+360. for x in lons_left]
        ar_full=xr.concat([ar_right,ar,ar_left],dim='lon')

        if i==0 and j==0:
            lon_array=np.zeros((ar_full.lat.size,ar_full.lon.size))
            lat_array=np.zeros((ar_full.lat.size,ar_full.lon.size))
            for j1 in range(ar_full.lat.size):
                lon_array[j1,:]=ar_full.lon.values
            for j1 in range(ar_full.lon.size):
                lat_array[:,j1]=ar_full.lat.values

        if j%100==0:
            print(j)

        ar_sel=ar_full.values
        structure=np.ones((3,3))
        ar_labeled,n=label(ar_sel,structure)
        ar_labeled=np.where(ar_sel==1,ar_labeled,np.nan)
        
        if 1==1:
            labeled_final=np.zeros_like(ar_labeled)
            elim_list=[]
            for k in range(n+1):
                ones=np.where(ar_labeled==k,1,0)
                num=np.nansum(ones)
                if num>0:
                    #find corresponding one
                    
                    lon_check=np.where(ar_labeled==k,lon_array,np.nan)
                    lat_check=np.where(ar_labeled==k,lat_array,np.nan)
                    lon_check_min=np.nanmin(lon_check)
                    lon_check_max=np.nanmax(lon_check)

                    lons=lon_check.flatten().tolist()
                    lats=lat_check.flatten().tolist()
                    lons=[x for x in lons if x>=-1e3]
                    lats=[x for x in lats if x>=-1e3]

                    zipped_latlon=list(zip(lons,lats))
                    select_lon_min=[x[0] for x in zipped_latlon if x[0]==lon_check_min][0]
                    select_lat_min=[x[1] for x in zipped_latlon if x[0]==lon_check_min][0]
                    select_lon_max=[x[0] for x in zipped_latlon if x[0]==lon_check_max][0]
                    select_lat_max=[x[1] for x in zipped_latlon if x[0]==lon_check_max][0]

                    a=np.unique(lon_check)
                    b=len([x for x in a if x>-1000])
                    if (lon_check_min<-180. and lon_check_max>=-180. or lon_check_min<180 and lon_check_max>=180) and b<960:
                        if k not in elim_list:
                            if lon_check_min<-180:
                                alternate_value=np.where(lat_array==select_lat_min,ar_labeled,np.nan)
                                alternate_value=np.where(lon_array==select_lon_min+360.,alternate_value,np.nan)
                                alternate_value=[x for x in np.unique(alternate_value) if x>=0]
                                if len(alternate_value)>1:
                                    print('ERROR')
                                    sys.exit()
                                else:
                                    alternate_value=alternate_value[0]
                                    
                            if lon_check_max>=180:
                                alternate_value=np.where(lat_array==select_lat_max,ar_labeled,np.nan)
                                alternate_value=np.where(lon_array==select_lon_max-360.,alternate_value,np.nan)
                                alternate_value=[x for x in np.unique(alternate_value) if x>=0]
                                if len(alternate_value)>1:
                                    print('ERROR')
                                    sys.exit()
                                else:
                                    alternate_value=alternate_value[0]
                                    
                            select_component1=np.where(ar_labeled==alternate_value,k,0)
                            labeled_final=labeled_final+ones*k+select_component1
                            elim_list.append(alternate_value)
        
                    else:
                        if k not in elim_list:
                            labeled_final=labeled_final+ones*k

            labeled_final1=xr.zeros_like(ar_full)
            labeled_final1[:,:]=labeled_final
            labeled_final=labeled_final1.where(labeled_final1>0)
            lons_center=[x for x in labeled_final.lon.values if -180.<=x<180.]
            labeled_final1=labeled_final1.sel(lon=lons_center)

        values=np.unique(labeled_final1)
        values=[x for x in values if x>0]
        num_total=num_total+len(values)
        
        field_new=ds1['ar_labeled'][j,:,:]
        values=np.unique(field_new)
        values=[x for x in values if x>0]
        num_new=num_new+len(values)
        
    print((num_total,num_new))

In [ ]:
#yrs=np.arange(1940,1941,1)
yrs=np.arange(2004,2020,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year) 
    #ds=xr.open_dataset(dir2+'era_5_ar_detection_polar_'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra_2_ar_detection_'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era_5_ar_detection_test_'+str(year)+'.nc')
    
    ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir1+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    
    for j in range(ds.time.size):
        ar=ds['ar'][j,:,:]
 
        lons_right=[x for x in ar.lon.values if x>=60.]
        lons_left=[x for x in ar.lon.values if x<-60.]
        ar_right=ar.sel(lon=lons_right)
        ar_left=ar.sel(lon=lons_left)
        ar_right['lon']=[x-360. for x in lons_right]
        ar_left['lon']=[x+360. for x in lons_left]
        ar_full=xr.concat([ar_right,ar,ar_left],dim='lon')

        if i==0 and j==0:
            lon_array=np.zeros((ar_full.lat.size,ar_full.lon.size))
            lat_array=np.zeros((ar_full.lat.size,ar_full.lon.size))
            for j1 in range(ar_full.lat.size):
                lon_array[j1,:]=ar_full.lon.values
            for j1 in range(ar_full.lon.size):
                lat_array[:,j1]=ar_full.lat.values

        if j%100==0:
            print(j)

        ar_sel=ar_full.values
        structure=np.ones((3,3))
        ar_labeled,n=label(ar_sel,structure)
        ar_labeled=np.where(ar_sel==1,ar_labeled,np.nan)
        
        if 1==1:
            labeled_final=np.zeros_like(ar_labeled)
            elim_list=[]
            for k in range(n+1):
                ones=np.where(ar_labeled==k,1,0)
                num=np.nansum(ones)
                if num>=100:
                    #find corresponding one
                    
                    lon_check=np.where(ar_labeled==k,lon_array,np.nan)
                    lat_check=np.where(ar_labeled==k,lat_array,np.nan)
                    lon_check_min=np.nanmin(lon_check)
                    lon_check_max=np.nanmax(lon_check)

                    lons=lon_check.flatten().tolist()
                    lats=lat_check.flatten().tolist()
                    lons=[x for x in lons if x>=-1e3]
                    lats=[x for x in lats if x>=-1e3]

                    zipped_latlon=list(zip(lons,lats))
                    select_lon_min=[x[0] for x in zipped_latlon if x[0]==lon_check_min][0]
                    select_lat_min=[x[1] for x in zipped_latlon if x[0]==lon_check_min][0]
                    select_lon_max=[x[0] for x in zipped_latlon if x[0]==lon_check_max][0]
                    select_lat_max=[x[1] for x in zipped_latlon if x[0]==lon_check_max][0]

                    a=np.unique(lon_check)
                    b=len([x for x in a if x>-1000])
                    if (lon_check_min<-180. and lon_check_max>=-180. or lon_check_min<180 and lon_check_max>=180) and b<960:
                        if k not in elim_list:
                            if lon_check_min<-180:
                                alternate_value=np.where(lat_array==select_lat_min,ar_labeled,np.nan)
                                alternate_value=np.where(lon_array==select_lon_min+360.,alternate_value,np.nan)
                                alternate_value=[x for x in np.unique(alternate_value) if x>=0]
                                if len(alternate_value)>1:
                                    print('ERROR')
                                    sys.exit()
                                else:
                                    alternate_value=alternate_value[0]
                                    
                            if lon_check_max>=180:
                                alternate_value=np.where(lat_array==select_lat_max,ar_labeled,np.nan)
                                alternate_value=np.where(lon_array==select_lon_max-360.,alternate_value,np.nan)
                                alternate_value=[x for x in np.unique(alternate_value) if x>=0]
                                if len(alternate_value)>1:
                                    print('ERROR')
                                    sys.exit()
                                else:
                                    alternate_value=alternate_value[0]
                                    
                            select_component1=np.where(ar_labeled==alternate_value,k,0)
                            labeled_final=labeled_final+ones*k+select_component1
                            elim_list.append(alternate_value)
        
                    else:
                        if k not in elim_list:
                            labeled_final=labeled_final+ones*k

            labeled_final1=xr.zeros_like(ar_full)
            labeled_final1[:,:]=labeled_final
            labeled_final=labeled_final1.where(labeled_final1>0)
            lons_center=[x for x in labeled_final.lon.values if -180.<=x<180.]
            labeled_final1=labeled_final1.sel(lon=lons_center)

        if j==0:
            labeled_concat=labeled_final1
        else:
            labeled_concat=xr.concat([labeled_concat,labeled_final1],dim='time')
        
    dk=xr.Dataset()
    dk['ar_labeled']=(('time','lat','lon'),labeled_concat.values)
    dk.coords['time']=labeled_concat.time
    dk.coords['lat']=labeled_concat.lat
    dk.coords['lon']=labeled_concat.lon
    try:
        os.remove(dir2+'era5tp.ar.labels.model.'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir2+'era5tp.ar.labels.model.'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()
print('DONE')

In [ ]:
yrs=np.arange(2023,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir+'merra_2_ar_detection_labeled_'+str(year)+'.nc')
    print(ds)
    labeled=ds['ar_labeled']
    ar_der=labeled/labeled
    ar_der=ar_der.fillna(0)
    
    dk=xr.Dataset()
    dk['ar']=(('time','lat','lon'),ar_der.values)
    dk.coords['time']=ar_der.time
    dk.coords['lat']=ar_der.lat
    dk.coords['lon']=ar_der.lon
    
    try:
        os.remove(dir+'merra_2_ar_detection_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir+'merra_2_ar_detection_'+str(year)+'.nc',mode='w',format='NETCDF4')